This notebook is used to connect to the Llamastack server and do some basic testing

Let's start by querying the Llamastack server and check if it is healthy and ready to accept requests. First we need to install the llama-stack client using pip:

In [1]:
%pip install llama-stack-client==0.2.22 rich


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Now, import the `LlamaStackClient` class, and set the base URL of the Llamastack server (You can get the URL by running `oc get svc -n competitor-analysis`)

In [2]:
from llama_stack_client import LlamaStackClient
import rich

LS_URL = "http://llama-stack-dist-service:8321"

# For access from Notebooks external to the cluster, use the route URL instead (oc get route -n competitor-analysis)
# LS_URL = "https://llama-stack-ext-competitor-analysis.apps.cluster-x5jfr.x5jfr.sandbox2053.opentlc.com"

# Initialize the client
client = LlamaStackClient(base_url=LS_URL)

List the models available in this Llamastack instance.

In [3]:
models = client.models.list()
rich.print(models)

INFO:httpx:HTTP Request: GET http://llama-stack-dist-service:8321/v1/models "HTTP/1.1 200 OK"


[
    Model(
        identifier='granite-embedding-125m',
        metadata={'embedding_dimension': 768.0},
        api_model_type='embedding',
        provider_id='sentence-transformers',
        type='model',
        provider_resource_id='ibm-granite/granite-embedding-125m-english',
        model_type='embedding'
    ),
    Model(
        identifier='vllm-inference/granite-3-3-8b-instruct',
        metadata={},
        api_model_type='llm',
        provider_id='vllm-inference',
        type='model',
        provider_resource_id='granite-3-3-8b-instruct',
        model_type='llm'
    ),
    Model(
        identifier='sentence-transformers/all-MiniLM-L6-v2',
        metadata={'embedding_dimension': 384.0},
        api_model_type='embedding',
        provider_id='sentence-transformers',
        type='model',
        provider_resource_id='all-MiniLM-L6-v2',
        model_type='embedding'
    )
]

Now, list the providers available in this Llamastack instance

In [4]:
providers = client.providers.list()
rich.print(providers)

INFO:httpx:HTTP Request: GET http://llama-stack-dist-service:8321/v1/providers "HTTP/1.1 200 OK"


[
    ProviderInfo(
        api='inference',
        config={
            'url': 
'https://granite-3-3-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1',
            'max_tokens': 4096.0,
            'api_token': '********',
            'tls_verify': False
        },
        health={'status': 'OK'},
        provider_id='vllm-inference',
        provider_type='remote::vllm'
    ),
    ProviderInfo(
        api='inference',
        config={},
        health={'status': 'Not Implemented', 'message': 'Provider does not implement health check'},
        provider_id='sentence-transformers',
        provider_type='inline::sentence-transformers'
    ),
    ProviderInfo(
        api='vector_io',
        config={
            'db_path': '/opt/app-root/src/.llama/distributions/rh/milvus.db',
            'kvstore': {
                'type': 'sqlite',
                'namespace': None,
                'db_path': '/opt/app-root/src/.llama/distributions/rh/milvus_registry.db'
            }
        },
        health={'status': 'Not Implemented', 'message': 'Provider does not implement health check'},
        provider_id='milvus',
        provider_type='inline::milvus'
    ),
    ProviderInfo(
        api='vector_io',
        config={
            'uri': 'http://milvus-service.competitor-analysis.svc.cluster.local:19530',
            'token': 'root:milvus123',
            'secure': None,
            'consistency_level': 'Bounded',
            'ca_pem_path': None,
            'client_pem_path': None,
            'client_key_path': None,
            'kvstore': {
                'type': 'sqlite',
                'namespace': None,
                'db_path': '/opt/app-root/src/.llama/distributions/rh/milvus_remote_registry.db'
            }
        },
        health={'status': 'Not Implemented', 'message': 'Provider does not implement health check'},
        provider_id='milvus-remote',
        provider_type='remote::milvus'
    ),
    ProviderInfo(
        api='safety',
        config={'orchestrator_url': None, 'ssl_cert_path': None, 'shields': {}},
        health={'status': 'Not Implemented', 'message': 'Provider does not implement health check'},
        provider_id='trustyai_fms',
        provider_type='remote::trustyai_fms'
    ),
    ProviderInfo(
        api='agents',
        config={
            'persistence_store': {
                'type': 'sqlite',
                'namespace': None,
                'db_path': '/opt/app-root/src/.llama/distributions/rh/agents_store.db'
            },
            'responses_store': {
                'type': 'sqlite',
                'db_path': '/opt/app-root/src/.llama/distributions/rh/responses_store.db'
            }
        },
        health={'status': 'Not Implemented', 'message': 'Provider does not implement health check'},
        provider_id='meta-reference',
        provider_type='inline::meta-reference'
    ),
    ProviderInfo(
        api='eval',
        config={
            'use_k8s': True,
            'base_url': 
'https://granite-3-3-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1'
        },
        health={'status': 'Not Implemented', 'message': 'Provider does not implement health check'},
        provider_id='trustyai_lmeval',
        provider_type='remote::trustyai_lmeval'
    ),
    ProviderInfo(
        api='datasetio',
        config={
            'kvstore': {
                'type': 'sqlite',
                'namespace': None,
                'db_path': '/opt/app-root/src/.llama/distributions/rh/huggingface_datasetio.db'
            }
        },
        health={'status': 'Not Implemented', 'message': 'Provider does not implement health check'},
        provider_id='huggingface',
        provider_type='remote::huggingface'
    ),
    ProviderInfo(
        api='datasetio',
        config={
            'kvstore': {
                'type': 'sqlite',
                'namespace': None,
                'db_path': '/opt/app-root/src/

Let's send a simple prompt to the Granite model running on MaaS via the Llamastack API

In [5]:
inference_model_id = next(m.identifier for m in models if m.model_type == "llm") # Get the Granite model for inference

prompt = "What is the capital of Mongolia?"

response = client.inference.chat_completion(
        messages=[{"role": "user", "content": prompt}],
        model_id=inference_model_id
    )
rich.print(response)

/tmp/ipykernel_3472/1625813307.py:5: DeprecationWarning: /v1/inference/chat-completion is deprecated. Please use /v1/openai/v1/chat/completions.
  response = client.inference.chat_completion(
INFO:httpx:HTTP Request: POST http://llama-stack-dist-service:8321/v1/inference/chat-completion "HTTP/1.1 200 OK"


ChatCompletionResponse(
    completion_message=CompletionMessage(
        content="The capital of Mongolia is Ulaanbaatar. Established in 1639, Ulaanbaatar has grown to become the 
country's political, economic, and cultural hub. It is the largest city in Mongolia, with a population of over 1.4 
million people as of 2021. The city is situated in the north-central part of the country, nestled among the 
foothills of the Mongolian-Manchurian mountain range, near the Tuul River. Ulaanbaatar serves as the seat of the 
government, home to numerous ministries, the President's residence, and the Mongolian parliament, the State Great 
Khural. The city is also a significant transportation center, connecting various regions of Mongolia and serving as
a gateway to the country's rich history, nomadic culture, and stunning natural landscapes.",
        role='assistant',
        stop_reason='end_of_turn',
        tool_calls=[]
    ),
    logprobs=None,
    metrics=[
        Metric(metric='prompt_tokens', value=17.0, unit=None),
        Metric(metric='completion_tokens', value=177.0, unit=None),
        Metric(metric='total_tokens', value=194.0, unit=None)
    ]
)